<font size=4>
    Here is a step by step, more controlled implementation of URF protocol.
    <br><br>
</font>

### Requirements <br>
<font size=4>
    X - input data <br>
</font>
<font size=3>
    &emsp; a high dimensional 2d numpy array <br>
    &emsp; if input data from multiple MD trajectories, then X has to be concatednated <br>
    &emsp; in the original work, only continuous data was tested, coming from internal coordiantes of proteins via MD simulations <br><br>
</font>

In [1]:
import numpy as np

In [5]:
X = np.load('../data/input_data.npy')
X.shape

(159729, 3741)

### URF

In [2]:
import sys
sys.path.append('../URF/')

In [9]:
import synthetic_data as syn
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split as splt
import scipy.cluster as scc
import proximity_matrix as pmt
import utils
import cluster
import metrics

#### synthetic data

In [6]:
dsyn = syn.synthetic_data(X, size=1)
dsyn.permute()   # may be dsyn.{marginal, nonsense, random etc}, nonsense indicate fictitious
pfeatures, plabels = create_permute.get_output()

In [7]:
# pfeatures and plabels are synthetic data+real data and artificial labels

#### RF1

In [8]:
np.random.seed(0)
rf1_cv=5
random_states = np.random.randint(0,1000,rf1_cv)

In [ ]:
clfs=[]
for i in range(rf1_cv):
    clf = rfc(n_estimators=1000, random_state=random_states[i])
    x,_,y,_ = splt(pfeatures, plabels, train_size=0.3, random_state=random_states[i])
    clf.fit(x,y)
    clfs.append(clf)

#### proximity estimations

In [11]:
prox = pmt.calculate_condenced_pmt_(clfs, X, n_jobs=2)
# here the size of X can be controlled, as per efficient scheme
# different schemes are available in proximity_matrix.py and cluster.py

#### hierarchical classes

In [13]:
hc = scc.hierarchy.linkage(prox, method='average')
# low-mem and fit-predict clustering schemes are available in cluster.py

In [14]:
nhc=2
Y = utils.get_hc_dtraj(hc, nids=nhc)

#### RF2

In [15]:
np.random.seed(0)
rf2_cv=5
random_states = np.random.randint(0,1000,rf2_cv)

In [18]:
fimp = np.zeros((rf2_cv, X.shape[1]))
LC = np.zeros((rf2_cv))
for i in range(len(rf2_cv)):
    clf = rfc(n_estimators=1000, n_jobs=2, random_state=random_states[i])
    xtrain, xtest, ytrain, ytest = splt(X, Y, test_size=0.3, random_state=random_states[i])
    clf.fit(xtrain, ytrain)
    fimp[i]=clf.feature_importances_
    LC[i] = metrics.learning_coefficient(Y, clf, xtest, ytest,
                                        cfactor=2, b=0, a=1, penalty=0)

### the fimp and LC are output